# Example Model Servers with Seldon

## Setup Seldon Core

Follow the instructions to [Setup Cluster](seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](seldon_core_setup.ipynb#Install-Seldon-Core).

## Serve SKlearn Iris Model

In [5]:
!pygmentize ../servers/sklearnserver/samples/iris.yaml

apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: sklearn
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    name: default
    replicas: 1
    svcOrchSpec: 
      env: 
      - name: SELDON_LOG_LEVEL
        value: DEBUG


In [6]:
!kubectl apply -f ../servers/sklearnserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io/sklearn created


In [7]:
!kubectl rollout status deploy/iris-default-4903e3c

Waiting for deployment "iris-default-4903e3c" rollout to finish: 0 of 1 updated replicas are available...
deployment "iris-default-4903e3c" successfully rolled out


In [1]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="sklearn",namespace="seldon")

In [2]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,4))
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 4
    values: 0.5532809728679123
    values: 0.4342251063258179
    values: 0.06149633665408394
    values: 0.4239944325524071
  }
}

Response:
meta {
  puid: "vf7s9q31lojvtoou0s593o2amq"
  requestPath {
    key: "classifier"
    value: "seldonio/sklearnserver_rest:0.2"
  }
}
data {
  names: "t:0"
  names: "t:1"
  names: "t:2"
  tensor {
    shape: 1
    shape: 3
    values: 0.48054487796869
    values: 0.39136042893422457
    values: 0.12809469309708543
  }
}



In [3]:
!kubectl delete -f ../servers/sklearnserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io "sklearn" deleted


## Serve XGBoost Iris Model

In [8]:
!pygmentize ../servers/xgboostserver/samples/iris.yaml

apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: xgboost
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: XGBOOST_SERVER
      modelUri: gs://seldon-models/xgboost/iris
      name: classifier
    name: default
    replicas: 1


In [9]:
!kubectl apply -f ../servers/xgboostserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io/xgboost created


In [10]:
!kubectl rollout status deploy/iris-default-af1783b

Waiting for deployment "iris-default-af1783b" rollout to finish: 0 of 1 updated replicas are available...
deployment "iris-default-af1783b" successfully rolled out


In [11]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="xgboost",namespace="seldon")

In [12]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,4))
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 4
    values: 0.8515452360370743
    values: 0.5935107411486708
    values: 0.5544182060177882
    values: 0.9355298659457035
  }
}

Response:
meta {
  puid: "u1vhv4n378m1ai5uu5iavp5b53"
  requestPath {
    key: "classifier"
    value: "seldonio/xgboostserver_rest:0.2"
  }
}
data {
  tensor {
    shape: 1
    values: 0.0
  }
}



In [13]:
!kubectl delete -f ../servers/xgboostserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io "xgboost" deleted


## Serve Tensorflow MNIST Model

**Will only work on a GCP Kubernetes Cluster**

In [14]:
!pygmentize ../servers/tfserving/samples/mnist_rest.yaml

apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: tfserving
spec:
  name: mnist
  predictors:
  - graph:
      children: []
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/mnist-model
      name: mnist-model
      parameters:
        - name: signature_name
          type: STRING
          value: predict_images
        - name: model_name
          type: STRING
          value: mnist-model
    name: default
    replicas: 1


In [15]:
!kubectl apply -f ../servers/tfserving/samples/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io/tfserving created


In [19]:
!kubectl rollout status deploy/mnist-default-725903e

deployment "mnist-default-725903e" successfully rolled out


In [20]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="tfserving",namespace="seldon")

In [21]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,784))
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 784
    values: 0.9077521075865232
    values: 0.481494282768426
    values: 0.4326137508614173
    values: 0.39412542521520766
    values: 0.2540870260101048
    values: 0.7407421531831561
    values: 0.9075359591452956
    values: 0.522853882249715
    values: 0.1544018196830882
    values: 0.2037192050420561
    values: 0.677923447833067
    values: 0.7171500065299637
    values: 0.08749274899503934
    values: 0.7355149948410137
    values: 0.1874174160636044
    values: 0.2909706448666053
    values: 0.3418987032626096
    values: 0.3329337836373806
    values: 0.8434577191142036
    values: 0.4861160253821838
    values: 0.8318209015954431
    values: 0.8976272369790321
    values: 0.9583714018073152
    values: 0.9528392802914424
    values: 0.9071597834656283
    values: 0.9753120098510758
    values: 0.5555712906817557
    values: 0.5346651649789239
    values: 0.09851330244536882
    values: 0.6664706250

In [22]:
!kubectl delete -f ../servers/tfserving/samples/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io "tfserving" deleted


## Serve MLFlow Elasticnet Wines Model

In [31]:
!pygmentize ../servers/mlflowserver/samples/elasticnet_wine.yaml

apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: mlflow
spec:
  name: wines
  predictors:
  - graph:
      children: []
      implementation: MLFLOW_SERVER
      modelUri: gs://seldon-models/mlflow/elasticnet_wine
      name: classifier
    name: default
    replicas: 1


In [32]:
!kubectl apply -f ../servers/mlflowserver/samples/elasticnet_wine.yaml

seldondeployment.machinelearning.seldon.io/mlflow created


In [33]:
!kubectl rollout status deploy/wines-default-8c791aa

Waiting for deployment "wines-default-8c791aa" rollout to finish: 0 of 1 updated replicas are available...
deployment "wines-default-8c791aa" successfully rolled out


In [34]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mlflow",namespace="seldon")

In [36]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,11))
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 11
    values: 0.6907478776933986
    values: 0.8309510656778891
    values: 0.562660723739024
    values: 0.016727865284311028
    values: 0.2935636350018127
    values: 0.6049700606468156
    values: 0.26904374522513186
    values: 0.059964759754932206
    values: 0.11900130537727505
    values: 0.30425941706369986
    values: 0.42276301953693896
  }
}

Response:
meta {
  puid: "kh2v4blpf1fh8c91o7jl4ucocr"
  requestPath {
    key: "classifier"
    value: "seldonio/mlflowserver_rest:0.2"
  }
}
data {
  tensor {
    shape: 1
    values: 5.212350520219259
  }
}



In [37]:
!kubectl delete -f ../servers/mlflowserver/samples/elasticnet_wine.yaml

seldondeployment.machinelearning.seldon.io "mlflow" deleted
